In [2]:
import pandas as pd
import folium
from pyproj import Transformer

# Stap 1: Laad de dataset
df = pd.read_csv("WKP_Meetobjecten_Waterschap Brabantse Delta_2024_20250414172129.csv", sep=";", encoding="utf-8")

# Stap 2: Zet RD naar WGS84
transformer = Transformer.from_crs("EPSG:28992", "EPSG:4326", always_xy=True)
df[['lon', 'lat']] = df.apply(
    lambda row: pd.Series(transformer.transform(row['GeometriePuntX_RD'], row['GeometriePuntY_RD'])),
    axis=1
)

# Stap 3: Definieer voedselbossen (al in RD → omzetten naar lat/lon)
voedselbossen_rd = {
    "Vierhoeven Roosendaal": (94471, 408274),
    "Haverleij": (136282, 412233),
    "Kleindongen": (116648, 407281),
    "Wildernest": (131417, 383390)
}
voedselbossen_wgs = {
    naam: transformer.transform(x, y)
    for naam, (x, y) in voedselbossen_rd.items()
}

# Stap 4: Maak de kaart
m = folium.Map(location=[51.6, 5.0], zoom_start=9)

# Voeg meetpunten toe
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=3,
        color='blue',
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# Voeg voedselbossen toe
for naam, (lon, lat) in voedselbossen_wgs.items():
    folium.Marker(
        location=[lat, lon],
        popup=naam,
        icon=folium.Icon(color='green', icon='leaf')
    ).add_to(m)

# Laat kaart zien (in Jupyter gebruik: m)
m.save("meetpunten_voedselbossen_kaart.html")
print("Kaart opgeslagen als 'meetpunten_voedselbossen_kaart.html'")


Kaart opgeslagen als 'meetpunten_voedselbossen_kaart.html'


In [3]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KDTree
import os

# Pad naar bestanden
bestand_pad = "Meetpunten"

# Bestanden van 2016 t/m 2024
bestanden_vanaf_2016 = [
    "WKP_Meetobjecten_Waterschap Brabantse Delta_2016_20250414192547.csv",
    "WKP_Meetobjecten_Waterschap Brabantse Delta_2017_20250414192542.csv",
    "WKP_Meetobjecten_Waterschap Brabantse Delta_2018_20250414192538.csv",
    "WKP_Meetobjecten_Waterschap Brabantse Delta_2019_20250414192533.csv",
    "WKP_Meetobjecten_Waterschap Brabantse Delta_2020_20250414192527.csv",
    "WKP_Meetobjecten_Waterschap Brabantse Delta_2021_20250414192523.csv",
    "WKP_Meetobjecten_Waterschap Brabantse Delta_2022_20250414192519.csv",
    "WKP_Meetobjecten_Waterschap Brabantse Delta_2023_20250414192514.csv",
    "WKP_Meetobjecten_Waterschap Brabantse Delta_2024_20250414172129.csv"
]

# Coördinaten uit elk bestand in sets
coordinaten_sets_2016 = {
    bestand: set(zip(
        pd.read_csv(os.path.join(bestand_pad, bestand), sep=';', encoding='utf-8')['GeometriePuntX_RD'],
        pd.read_csv(os.path.join(bestand_pad, bestand), sep=';', encoding='utf-8')['GeometriePuntY_RD']
    ))
    for bestand in bestanden_vanaf_2016
}

# Zet coördinatensets om in lijsten en arrays
coordinaten_lijsten_2016 = [np.array(list(coords)) for coords in coordinaten_sets_2016.values()]
eerste_jaar_coords = coordinaten_lijsten_2016[0]
overige_jaren = coordinaten_lijsten_2016[1:]

# Maak KDTree voor alle overige jaren
bomen = [KDTree(coords) for coords in overige_jaren]

# Checkfunctie binnen radius
def komt_voor_in_alle_jaren(coord, bomen, radius=200):
    for boom in bomen:
        afstanden, _ = boom.query_radius([coord], r=radius, return_distance=True)
        if len(afstanden[0]) == 0:
            return False
    return True

# Selecteer gemeenschappelijke coördinaten
gemeenschappelijke_coords_200m_2016 = [
    tuple(coord) for coord in eerste_jaar_coords if komt_voor_in_alle_jaren(coord, bomen)
]

# Print resultaten
print(f"Aantal locaties die in alle jaren voorkomen (binnen 200 meter): {len(gemeenschappelijke_coords_200m_2016)}\n")
print("Voorbeelden:")
for coord in gemeenschappelijke_coords_200m_2016[:5]:
    print(coord)


Aantal locaties die in alle jaren voorkomen (binnen 200 meter): 15

Voorbeelden:
(78850.0, 381814.0)
(77128.0, 389226.0)
(110431.0, 414683.0)
(80718.0, 395721.0)
(75570.0, 387830.0)


In [4]:
import pandas as pd
import folium
from pyproj import Transformer

# Stap 1: Gebruik de gemeenschappelijke meetpunten-coördinaten uit 2016–2024 (binnen 200m)
gemeenschappelijke_coords_200m_2016 = [
    (78850.0, 381814.0),
    (77128.0, 389226.0),
    (110431.0, 414683.0),
    (80718.0, 395721.0),
    (75570.0, 387830.0),
    (85930.0, 393478.0),
    (94995.0, 399050.0),
    (79260.0, 397139.0),
    (109810.0, 401457.0),
    (98378.0, 414924.0),
    (113066.0, 406293.0),
    (87056.0, 385722.0),
    (98045.0, 417149.0),
    (95250.0, 395327.0),
    (102939.0, 417813.0)
]

# Stap 2: Zet RD naar WGS84
transformer = Transformer.from_crs("EPSG:28992", "EPSG:4326", always_xy=True)
meetpunten_wgs = [transformer.transform(x, y) for x, y in gemeenschappelijke_coords_200m_2016]

# Stap 3: Voedselbossen (bekend in RD)
voedselbossen_rd = {
    "Vierhoeven Roosendaal": (94471, 408274),
    "Haverleij": (136282, 412233),
    "Kleindongen": (116648, 407281),
    "Wildernest": (131417, 383390)
}
voedselbossen_wgs = {
    naam: transformer.transform(x, y)
    for naam, (x, y) in voedselbossen_rd.items()
}

# Stap 4: Maak kaart
m = folium.Map(location=[51.6, 5.0], zoom_start=9)

# Voeg gemeenschappelijke meetpunten toe
for lon, lat in meetpunten_wgs:
    folium.CircleMarker(
        location=[lat, lon],
        radius=4,
        color='blue',
        fill=True,
        fill_opacity=0.8
    ).add_to(m)

# Voeg voedselbossen toe
for naam, (lon, lat) in voedselbossen_wgs.items():
    folium.Marker(
        location=[lat, lon],
        popup=naam,
        icon=folium.Icon(color='green', icon='leaf')
    ).add_to(m)

# Opslaan
m.save("/mnt/data/kaart_meetpunten_2016_2024_met_voedselbossen.html")
"/mnt/data/kaart_meetpunten_2016_2024_met_voedselbossen.html"


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/kaart_meetpunten_2016_2024_met_voedselbossen.html'